# Using ROOT to bind Python and C++

PyROOT is not just a set of Python bindings to C++ functions; it dynamically generates bindings for all C++ classes and functions in its namespace.

Including new functions that you define.

In [3]:
import ROOT

In a script, use `gInterpreter.ProcessLine` or `gInterpreter.Declare` to run C++ code.

In [4]:
ROOT.gInterpreter.Declare("""
double dostuff(double a, double b) {
    return a + b;
}
""")

False

input_line_33:2:8: error: redefinition of 'dostuff'
double dostuff(double a, double b) {
       ^
input_line_32:2:8: note: previous definition is here
double dostuff(double a, double b) {
       ^


In [5]:
ROOT.dostuff(3.14, 100)

103.14

Suppose we want to change the `dostuff` function, common on a prompt or in a notebook.

In [6]:
ROOT.gInterpreter.Declare("""
double dostuff(double a, double b) {
    return (a + b) / sqrt(a*a + b*b);
}
""")

False

input_line_35:2:8: error: redefinition of 'dostuff'
double dostuff(double a, double b) {
       ^
input_line_32:2:8: note: previous definition is here
double dostuff(double a, double b) {
       ^


Ouch! Of course it doesn't let us replace a definition (not legal in C++), but we really want to.

This is what I do:

In [7]:
tmpname = "dostuffv1"                  # keep changing the C++ name

ROOT.gInterpreter.Declare("""
double %s(double a, double b) {
    return (a + b) / sqrt(a*a + b*b);
}
""" % tmpname)

dostuff = getattr(ROOT, tmpname)       # but assign to the same Python name

In [8]:
dostuff(3.14, 100)

1.0308919161099395

<center><img src="images/03-cheat-sheet.png"></center>

## ROOT and Numpy arrays

Passing between C++ and Python is an expensive process (looking up class and function definitions in ROOT's C++ reflection and dynamically generating equivalent Python structures).

To use these bindings efficiently, we should send bulk data over the boundary and do loops on the C++ side.

Python's bulk container is the Numpy array.

In [9]:
import numpy
a = numpy.arange(10) * 1.1
a

array([0. , 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [10]:
tmpname = "dostuffv2"

ROOT.gInterpreter.Declare("""
void %s(int num, double *data) {
    for (int i = 0;  i < num;  i++) {
        data[i] += 100;
    }
}
""" % tmpname)

dostuff = getattr(ROOT, tmpname)

In [11]:
dostuff(len(a), a)
a

array([100. , 101.1, 102.2, 103.3, 104.4, 105.5, 106.6, 107.7, 108.8,
       109.9])

This works because ROOT converts the Numpy array of doubles into a C-style array of doubles.

But beware! It doesn't _check_ the array type, it _assumes_ the type.

In [12]:
b = numpy.arange(10, dtype=numpy.int64)    # same number of bits as double, but different meaning
b

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
dostuff(len(b), b)
b

array([4636737291354636288, 4636737291354636288, 4636737291354636288,
       4636737291354636288, 4636737291354636288, 4636737291354636288,
       4636737291354636288, 4636737291354636288, 4636737291354636288,
       4636737291354636288])

It gets even worse if the length is wrong: segmentation faults!

## ROOT and std::vectors

Modern C++ uses `std::vector` to protect both the length and the type, but the data are still contiguous in memory like a Numpy array.

ROOT provides a way to create `std::vectors` in Python and view them as Numpy.

In [14]:
a = ROOT.std.vector("double")()    # allocate empty
a.push_back(1.1)
a.push_back(2.2)
a.push_back(3.3)
print([x for x in a])

[1.1, 2.2, 3.3]


In [15]:
a_np = numpy.asarray(a)
a_np

array([1.1, 2.2, 3.3])

This is a _view_ of the _same memory._

In [16]:
a_np += 100
a_np

array([101.1, 102.2, 103.3])

In [17]:
print([x for x in a])

[101.1, 102.2, 103.3]


The Numpy array can even be unnamed and short-lived if we use `[:]` to set its data because it is _not_ a copy.

In [18]:
numpy.asarray(a)[1:] = 999
print([x for x in a])

[101.1, 999.0, 999.0]


(See Numpy tutorials for more about `[:]`.)

Using `std::vector`, we can write safer functions.

In [19]:
tmpname = "dostuffv3"

ROOT.gInterpreter.Declare("""
std::vector<double> %s(std::vector<double> &a, std::vector<double> &b) {
    std::vector<double> out;
    for (int i = 0;  i < a.size() && i < b.size();  i++) {
        out.push_back(a[i] + b[i]);
    }
    return out;
}
""" % tmpname)

dostuff = getattr(ROOT, tmpname)

In [20]:
a = ROOT.std.vector("double")(10)
b = ROOT.std.vector("double")(10)
numpy.asarray(a)[:] = numpy.arange(10) * 1.1
numpy.asarray(b)[:] = 100.0

In [21]:
c = dostuff(a, b)
numpy.asarray(c)

array([100. , 101.1, 102.2, 103.3, 104.4, 105.5, 106.6, 107.7, 108.8,
       109.9])

Unlike Numpy arrays you create yourself, these arrays do not "own" their data. Memory leaks and `new/delete` must be handled entirely on the C++ side.

In [22]:
numpy.asarray(c).flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

## Coming soon!

<center><img src="images/03-coming-soon-1.png"></center>

<center><img src="images/03-coming-soon-2.png"></center>

<center><img src="images/03-coming-soon-3.png"></center>

<center><img src="images/03-coming-soon-4.png"></center>

<center><img src="images/03-coming-soon-5.png"></center>

<center><img src="images/03-coming-soon-6.png"></center>